In [ ]:
import os
import pandas as pd
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import chromosomeDefinition as cd
import analyseVCF as aVCF
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

In [ ]:
CNV_COLUMNS = ['chrom','posStart','posEnd','copyNum','gainOrLoss','BAF','score','germlineOrSomatic','oneOrZero']

In [ ]:
# LOAD BED
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
USE_BED = True
LOAD_FULL_FILE = True

### FUNCTIONS

In [ ]:
def findFiles(path,suffix):
    files=[]
    for x in os.listdir(path):
        if x[-len(suffix):] == suffix:
            files.append(x)
    return files

In [ ]:
def loadCNVforPatient(cnvFile,patientName):
    dfCNV = pd.read_table(cnvFile, names = CNV_COLUMNS )
    #add regions with default copy number
    last = dfCNV.iloc[0]
    for i in range(1, dfCNV.shape[0]-1):
        if last['posEnd']<>dfCNV.iloc[i]['posStart']:
            if last['chrom']==dfCNV.iloc[i]['chrom']:
                dfCNV.loc[len(dfCNV)] = [last['chrom'],last['posEnd'], dfCNV.iloc[i]['posStart']-1,2,'none','','',0,0]
            else:
                dfCNV.loc[len(dfCNV)] = [last['chrom'],last['posEnd'], cd.chromosomeLength[last['chrom']],2,'none','','',0,0]
                if dfCNV.iloc[i]['posStart']<>0:
                    dfCNV.loc[len(dfCNV)] = [dfCNV.iloc[i]['chrom'],0, dfCNV.iloc[i]['posStart'],2,'none','','',0,0]
        last = dfCNV.iloc[i]
    #fix first record
    if dfCNV.iloc[0]['posStart']<>0:
        dfCNV.loc[len(dfCNV)] = ['1',0, dfCNV.iloc[0]['posStart'],2,'none','','',0,0]
    #Additional Fields
    dfCNV['chromPos']= dfCNV['chrom'].apply(lambda x: cd.intChrom(x)) + dfCNV['posStart'] / dfCNV['chrom'].apply(lambda x: cd.chromosomeLength[str(x)])
    dfCNV['chrom'] = dfCNV['chrom'].apply(lambda x: cd.intChrom(x))
    dfCNV['cappedCopyNum'] = dfCNV['copyNum'].clip(upper=5)
    dfCNV = dfCNV.sort_values(["chrom","posStart","posEnd"]).reset_index()
    dfCNV['regionLength'] = (dfCNV['posEnd']-dfCNV['posStart'])
    dfCNV['patientName']=patientName
    return dfCNV

In [ ]:
def calculateMBAFBetween(chrom,minPos,maxPos):
    dftemp = dfBAF[(dfBAF.Chromosome==chrom)&(dfBAF.Position>minPos)&(dfBAF.Position<maxPos)]
    return dftemp['mBAF'].median(),dftemp['mBAF'].count()

In [ ]:
def loadBAFData(bafFile):
    df = pd.read_table(bafFile)
    return df

In [ ]:
def cnvChart(dfCNV,filename,minChrom=1,maxChrom=23):
    plt.figure(figsize=[18,3])
    plt.title(filename)
    ax = plt.gca()
    ax.plot(dfCNV['chromPos'], dfCNV['copyNum'],drawstyle="steps-post")
    ax.axis([minChrom, maxChrom, 0, 6])
       

In [ ]:
def cnvLabel(copyNumber):
    if copyNumber < 2:
        return 'loss'
    elif copyNumber > 2:
        return 'amplification'
    else:
        return'normal'

In [ ]:
def createBAF(dfGermline):
    dfBAF = dfGermline[(dfGermline['variantType']=="SNP")&(dfGermline['normalSGT']=="0/1")&(dfGermline['normalAF']>0.4) \
                   &(dfGermline['normalAF']<0.65)&(dfGermline['normalDP']>30)&\
                   (dfGermline['normalDP']<100)][['chrom','pos','tumorAF']]
    dfBAF.columns = ['Chromosome', 'Position','BAF']
    dfBAF['Position'] = pd.to_numeric(dfBAF['Position'])
    dfBAF['mBAF']= 0.5+abs(dfBAF['BAF']-0.5)
    dfBAF['chromPos']= dfBAF['Chromosome'].apply(lambda x: cd.intChrom(x)) + dfBAF['Position'] / dfBAF['Chromosome'].apply(lambda x: cd.chromosomeLength[str(x)])
    dfBAF['Chromosome']= dfBAF['Chromosome'].apply(lambda x: cd.intChrom(x))
    return dfBAF

In [ ]:
def patientIDFromFilename(filename,findKey):
    patientIDStart = filename.find(findKey)   #
    return filename[patientIDStart:patientIDStart+12]
    #return "GIAB12878"

In [ ]:
def germlineVCFSampleNames(filename,patientID):
    return {patientID+'R':'normal',patientID+'T':'tumor'}

In [ ]:
def loadVCF(path,filename,sampleNames,patientID,bedPath,bedFileName):
    df = pd.DataFrame()
    if USE_BED:
        bed = aVCF.loadBEDFile(bedPath,bedFileName)
        return pd.concat([df,aVCF.loadVariantsFromVCF(path,filename,sampleNames,patientID,True,True,bed,LOAD_FULL_FILE)])
    else:
        return pd.concat([df,aVCF.loadVariantsFromVCF(path,filename,sampleNames,patientID,True)])

In [ ]:
def AFByPosPlot(pos,AF,startChartPos,endChartPos,maxY=1.0,height=5):
    plt.figure(figsize=[18,height])
    plt.scatter(pos, AF)
    plt.grid(b=True, which='both', color='0.65',linestyle='-')
    plt.axis([startChartPos, endChartPos,0, maxY])
    plt.show()

In [ ]:
def loadPON(aPath,aPONFile):
    myPON = []
    with open(aPath + aPONFile, 'r') as f:
        for line in f:
            line = line.strip('\n')
            splitLine = line.split('\t')
            myPON.append(splitLine)
    dfPON = pd.DataFrame(myPON)
    dfPON.columns = ['chrom','pos','ref','alt','PONCount']
    return dfPON

In [ ]:
def findPeaks(log_dens):
    peaks = []
    troughs = []
    dens=np.exp(log_dens)
    diff = [dens[x] - dens[x-1] for x in range(1,len(dens))]
    for i in range (len(diff)-1):
        if diff[i+1] < 0 and diff[i] > 0:
            peaks.append(float(i+1)/len(dens))
        if diff[i+1] > 0 and diff[i] < 0:
            troughs.append(float(i+1)/len(dens))
    return peaks, troughs

In [ ]:
def pdfChart(log_dens,maxYValue=8):
    fig, ax = plt.subplots()
    ax.plot(X_plot[:, 0], np.exp(log_dens), '-',label="kernel = '{0}'".format('gaussian'))
    ax.legend(loc='upper right')
    fig.set_figwidth(10)
    fig.set_figheight(5)
    ax.set_xlim(0, 1)
    ax.set_ylim(-0.02, maxYValue)
    ax.grid(b=True, which='both', color='0.65',linestyle='-')
    plt.show()

In [ ]:
def calculateSomaticCNV(dfTumorCNV):
    lastSomaticCopyNum = 2
    dfTumorCNV['copyNumSomatic']= dfTumorCNV['copyNum']
    for i in range(1, dfTumorCNV.shape[0]-1):
        if dfTumorCNV.iloc[i].germlineOrSomatic=="germline":
                dfTumorCNV.ix[i,'copyNumSomatic'] = lastSomaticCopyNum
        elif dfTumorCNV.iloc[i].germlineOrSomatic=="somatic"  or dfTumorCNV.iloc[i].germlineOrSomatic=="-":  
            lastSomaticCopyNum = dfTumorCNV.ix[i,'copyNumSomatic']
    return dfTumorCNV

### LOAD PON

In [ ]:
# Only needs to be run once
#dfPON2 = loadPON("/Users/peterpriestley/hmf/analyses/PON/779filePON/","PON.tsv")

In [ ]:
def loadPONvcf(PONFile):
    numHeaderRows = 0
    with open(PONFile) as fp:
        while fp.readline()[0]=='#':
            numHeaderRows = numHeaderRows+1
    dfPON = pd.read_table(PONFile,skiprows =numHeaderRows-1, dtype={'#CHROM':'str','POS':'str'})  #names = CNV_COLUMNS
    dfPON['PON_COUNT'] = dfPON['INFO'].apply(lambda x: x.split('=')[1])
    dfPON.rename(columns={'#CHROM': 'chrom', 'POS': 'pos','REF':'ref','ALT':'alt'}, inplace=True)
    return dfPON

dfPON = loadPONvcf("/Users/peterpriestley/hmf/analyses/PON/PON.vcf")

### FIND ALL FILES

In [ ]:
PATH = "/Users/peterpriestley/hmf/analyses/pipelineV3/"

### TEMP

In [ ]:
somaticVCFFilename = findFiles(PATH,"melted.vcf")[0]
strelkaVCFFilename = findFiles(PATH,"processed.vcf")[0]
patientID = patientIDFromFilename(somaticVCFFilename,"CPCT0")
print somaticVCFFilename
print strelkaVCFFilename
print "patient =",patientID

In [ ]:
dfStrelka = loadVCF(PATH,strelkaVCFFilename,{patientID+'T':'melted'},patientID,BED_PATH,BED_FILE_NAME)
dfSomatic = loadVCF(PATH,somaticVCFFilename,{patientID+'T':'melted'},patientID,BED_PATH,BED_FILE_NAME)

In [ ]:
# APPLY PON to SOMATICs
dfStrelka['alt'] = dfStrelka['meltedallele']
dfStrelka['strelkaAF'] = dfStrelka['meltedAF']
dfStrelka = pd.merge(dfStrelka,dfPON,how='left', on=['chrom','pos','ref','alt'])
dfStrelka['PON_COUNT'].fillna(0, inplace=True)
dfStrelka['inPON'] = pd.to_numeric(dfStrelka.PON_COUNT,errors=coerce)>4

dfSomatic['alt'] = dfSomatic['meltedallele']
dfSomatic = pd.merge(dfSomatic,dfPON,how='left', on=['chrom','pos','ref','alt'])
dfSomatic['PON_COUNT'].fillna(0, inplace=True)
dfSomatic['inPON'] = pd.to_numeric(dfSomatic.PON_COUNT,errors=coerce)>4

In [ ]:
#dfStrelka['inSomatic'] = dfStrelka.meltedAF_y>0

In [ ]:
# Compare to SOMATICs
dfStrelka = pd.merge(dfStrelka,dfSomatic[dfSomatic.consensus==True][['chrom','pos','ref','alt','meltedAF']],how='left', on=['chrom','pos','ref','alt'])
dfStrelka['inSomatic'] = dfStrelka.meltedAF>0
dfSomatic = pd.merge(dfSomatic,dfStrelka[['chrom','pos','ref','alt','strelkaAF']],how='left', on=['chrom','pos','ref','alt'])
dfSomatic['inStrelka'] = dfSomatic.strelkaAF>0

In [ ]:
pd.pivot_table(dfStrelka[(((dfStrelka.strelkaQS>20)&(dfStrelka.strelkaAF>0.05))|(dfStrelka.bedRegion=='Default'))&(dfStrelka.inPON==False)&(dfStrelka.strelkaQS*dfStrelka.strelkaAF>1.3)&(dfStrelka.annAllEffects.str.contains('missense'))], values='pos', index=['variantType'], columns=['bedRegion'], aggfunc='count',margins=False).fillna("")

In [ ]:
pd.pivot_table(dfStrelka[(dfStrelka.inPON==False)], values='pos', index=['variantType','inSomatic'], columns=['bedRegion'], aggfunc='count',margins=False).fillna("")

In [ ]:
#pd.pivot_table(dfStrelka[(((dfStrelka.strelkaQS>20)&(dfStrelka.strelkaAF>0.1))|(dfStrelka.bedRegion=='Default'))&(dfStrelka.inPON==False)&(dfStrelka.strelkaQS*dfStrelka.strelkaAF>0)], values='pos', index=['inSomatic','variantType'], columns=['bedRegion'], aggfunc='count',margins=False).fillna("")

In [ ]:
#pd.pivot_table(dfStrelka[(dfStrelka.inPON==False)&(dfStrelka.strelkaQS*dfStrelka.strelkaAF>1)], values='pos', index=['inSomatic','variantType'], columns=['bedRegion'], aggfunc='count',margins=False).fillna("")

In [ ]:
#dfSomatic

In [ ]:
pd.pivot_table(dfSomatic[(dfSomatic.inPON==False)&(dfSomatic.consensus==True)], values='pos', index=['inStrelka','variantType'], columns=['bedRegion'], aggfunc='count',margins=False).fillna("")

### BED FILE ANALYSIS

In [ ]:
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
HCBed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)

In [ ]:
BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
BED_FILE_NAME = "CPCT_Slicing.bed"
hmfBed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
print sum(int(row[2])-int(row[1]) for row in hmfBed)

In [ ]:
HCItem = HCBed.pop()
HMFItem = hmfBed.pop()
overlap = 0
count = 0
prevOverlap,prevCount = 0,0
while HCBed and hmfBed: 
    if HCItem[0]=='X':
        HCItem[0] = 23
    if HCItem[0]=='Y':
        HCItem[0] = 24
    if HMFItem[0]=='X':
        HMFItem[0] = 23
    if HMFItem[0]=='Y':
        HMFItem[0] = 24
    if int(HCItem[0])<int(HMFItem[0]) or (int(HCItem[0])==int(HMFItem[0]) and int(HCItem[2]) < int(HMFItem[1])):
        #if int(HMFItem[0])>int(HCItem[0]):
        #    print HCItem
        print HMFItem[3],int(HMFItem[2])-int(HMFItem[1]),overlap-prevOverlap,round(float(overlap-prevOverlap)/(int(HMFItem[2])-int(HMFItem[1])),2),count-prevCount
        prevOverlap = overlap
        prevCount = count
        HMFItem = hmfBed.pop()
    elif int(HMFItem[0])<int(HCItem[0]) or (int(HCItem[0])==int(HMFItem[0]) and int(HMFItem[2]) < int(HCItem[1])):
        #if int(HMFItem[0])<int(HCItem[0]):
        #    print HMFItem
        HCItem = HCBed.pop()

    else:
        overlap = overlap + min(int(HCItem[2]),int(HMFItem[2])) - max(int(HCItem[1]),int(HMFItem[1]))
        count = count + 1
        #if count < 300: #HMFItem[3]=='ENST00000249373.3 (SMO)' or HMFItem[3] == 'ENST00000320356.2 (EZH2)':
         #   print HMFItem,HCItem,int(HMFItem[2])-int(HMFItem[1]),int(HCItem[2])-int(HCItem[1]),min(int(HCItem[2]),int(HMFItem[2])) - max(int(HCItem[1]),int(HMFItem[1]))
        if int(HCItem[1])<int(HMFItem[1]):
            print HMFItem[3],int(HMFItem[2])-int(HMFItem[1]),overlap-prevOverlap,round(float(overlap-prevOverlap)/(int(HMFItem[2])-int(HMFItem[1])),2),count-prevCount
            prevOverlap = overlap
            prevCount = count
            HMFItem = hmfBed.pop()
        else:
            
            HCItem = HCBed.pop()
                                
print overlap,count
            